In [38]:

# webscrape 20 images each of 10 characters
# save images to folder

import os
from bing_image_downloader import downloader
import shutil
import matplotlib.pyplot as plt

# create folder to save images if it doesn't exist
if not os.path.exists('images'):
    os.mkdir('images')
if not os.path.exists('imagestest'):
    os.mkdir('imagestest')

# temp folder incase any image in main folder gets deleted (reducing time taken while running)
if not os.path.exists('imageswork'):
    os.mkdir('imageswork')

In [39]:
# get list of characters from Assets 
file = open("Assets/characters.txt","r")
characters = []
for data in file:
    substr = '\n'
    if substr in data:
        data_i = data.split('\n')
        data = data_i[0]
    characters.append(data)
print(characters)

['Narendra Modi', 'Leonardo DiCaprio', 'Tom Cruise', 'Dhoni', 'Virat']


In [40]:
# %%script false --no-raise-error
# delete all folders which are not in characters
import cv2
for folder in os.listdir('images'):
    if folder not in characters:
        shutil.rmtree('images/' + folder)
# loop through each character
for character in characters:
    # download 20 images of each character
    downloader.download(character+" face", limit=15, output_dir='images/', adult_filter_off=True, force_replace=False, timeout=60, verbose=False,filter='photo')

for character in characters:
    if not os.path.exists('imageswork/'+ character+" face"):
        os.mkdir('imageswork/'+ character+" face")

for character in characters:
    for image in os.listdir('images/' + character+" face"):
        img = cv2.imread('images/' + character+" face" + '/' +image)
        cv2.imwrite('imageswork/'+ character+" face" + '/' + image, img)

[%] Downloading Images to c:\Users\aniru\datasets\MachineLearningCourse\Project\EigenFaces\images\Narendra Modi face
[!] Issue getting: https://www.celebrity-cutouts.co.uk/wp-content/uploads/2018/03/narendra-modi-celebrity-mask.png
[!] Error:: HTTP Error 404: Not Found
[!] Issue getting: https://i.guim.co.uk/img/media/73c69c9fef6527c98208df69ec8c1096c0f06469/0_116_3820_2292/master/3820.jpg?width=445&amp;quality=45&amp;auto=format&amp;fit=max&amp;dpr=2&amp;s=00ee438409358816771ea50ab9aa6d28
[!] Error:: HTTP Error 401: Unauthorized - missing signature
[!] Issue getting: http://www.suntiros.com/wp-content/uploads/2016/12/Narendra-Modi-1.jpg
[!] Error:: HTTP Error 403: Forbidden
[!] Issue getting: https://www.whoa.in/20140224-Whoa/Narendra-Modi-Smiley-Face-Closeup-with-Dark-Background-Wallpaper.jpg
[!] Error:: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1125)>


[%] Done. Downloaded 15 images.
[%] Downloadi

In [41]:
# create face only images

for character in characters:
    for image in os.listdir('imageswork/' + character+" face"):
        img = cv2.imread('imageswork/' + character+" face" + '/' +image)
        cv2.imwrite('images/'+ character + '/' + image, img)


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

# delete and recreate images_512_face folder
if os.path.exists('images_512_face'):
    shutil.rmtree('images_512_face')
os.mkdir('images_512_face')

for folder in os.listdir('images_512_face'):
    if folder not in characters:
        shutil.rmtree('images_512_face/' + folder)
for character in characters:
    if not os.path.exists('imagestest/'+ character+" face"):
        os.mkdir('imagestest/'+ character+" face")

# loop through each character
for character in characters:
    # create folder to save images if it doesn't exist
    if not os.path.exists('images_512_face/' + character):
        os.mkdir('images_512_face/' + character)
    # loop through each image
    for image in os.listdir('images/' + character+" face"):
        # read image
        imgOG = cv2.imread('images/' + character+" face" + '/' +image)
        #read img but as grayscale
        # cv2.IMREAD_GRAYSCALE
        img = cv2.imread('images/' + character+" face" + '/' +image, 0)
        # # Convert into grayscale
        # img = cv2.cvtColor(imgOG, cv2.COLOR_BGR2GRAY)
        # detect faces
        result = face_cascade.detectMultiScale3(img, 1.1, 4, outputRejectLevels=True)
        face=result[0]
        levelWeight=result[2]
        temp=None
        f=0
        #skip example if confidence not high enough
        while( len(face) == 0 or (levelWeight[0] < 20)):
            f=0
            # raise Exception("No face detected")
            # checking if face can be found, if no we change the image
            os.remove('images/' + character+" face" + '/'+image)
            shutil.rmtree('imagestest/' + character+" face/")
            downloader.download(character+" face", limit=1, output_dir='imagestest/', adult_filter_off=True, force_replace=False, timeout=60, verbose=False,filter='photo')
            for image1 in os.listdir('imagestest/' + character+" face"):
                img=cv2.imread('imagestest/' + character+" face" + '/'+image1)
                temp=img
                result = face_cascade.detectMultiScale3(img, 1.1, 4, outputRejectLevels=True)
                face=result[0]
                levelWeight=result[2]
                f=1
        if f==1:
            cv2.imwrite('images/'+ character+" face" + '/' + image, temp)

#crop face
for character in characters:
    for image in os.listdir('images/' + character+" face"):
        imgOG = cv2.imread('images/' + character+" face" + '/' +image)
        img = cv2.imread('images/' + character+" face" + '/' +image, 0)
        
        # Check if face read correctly
        if imgOG is None:
            continue
        result = face_cascade.detectMultiScale3(img, 1.1, 4, outputRejectLevels=True)
        face=result[0]
        for (x, y, w, h) in face:
            try:
                imgOG = imgOG[y:y+h, x:x+w]
            except:
                print("Face not properly detected")
                continue
        # resize image
        try:
            imgOG = cv2.resize(imgOG, (256, 256))
            cv2.imwrite('images_512_face/' + character + '/' + image, imgOG)
        except:
            print("Could not resize")
            continue
        # save image
        

shutil.rmtree('imagestest/')
shutil.rmtree('imageswork/')

[%] Downloading Images to c:\Users\aniru\datasets\MachineLearningCourse\Project\EigenFaces\imagestest\Narendra Modi face


[%] Done. Downloaded 1 images.
[%] Downloading Images to c:\Users\aniru\datasets\MachineLearningCourse\Project\EigenFaces\imagestest\Narendra Modi face


[%] Done. Downloaded 1 images.
[%] Downloading Images to c:\Users\aniru\datasets\MachineLearningCourse\Project\EigenFaces\imagestest\Narendra Modi face


[%] Done. Downloaded 1 images.
[%] Downloading Images to c:\Users\aniru\datasets\MachineLearningCourse\Project\EigenFaces\imagestest\Leonardo DiCaprio face
[!] Issue getting: https://www.partypeople.company/uploads/1/7/6/2/17627711/s454377924454347902_p98_i66_w826.jpeg
[!] Error:: HTTP Error 404: Not Found


[%] Done. Downloaded 1 images.
[%] Downloading Images to c:\Users\aniru\datasets\MachineLearningCourse\Project\EigenFaces\imagestest\Dhoni face


[%] Done. Downloaded 1 images.
[%] Downloading Images to c:\Users\aniru\datasets\MachineLearningCourse\Project\Ei

In [42]:
# create dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df=pd.DataFrame()
y=pd.DataFrame(columns=['character'])

def clamp(n, minn, maxn): 
    return max(min(maxn, n), minn)

# loop through every image
for character in characters:
    for image in os.listdir('images_512_face/' + character):
        # read image as grayscale
        img = cv2.imread('images_512_face/' + character + '/' + image, 0)
        #flatten image
        imgFlat = img.flatten()
        # add image to dataset
        df = pd.concat([df, pd.DataFrame(imgFlat).T], ignore_index=True)
        # add label to y
        y = pd.concat([y, pd.DataFrame([character], columns=['character'])], ignore_index=True)

In [43]:
# Lets combine the features and the label into single dataframe 
df_resultant = pd.concat([df,y],axis = 1)
print(df_resultant.head())

     0    1    2    3    4    5    6    7    8    9  ...  65527  65528  65529  \
0    4   23    4   46   20  116   35  100  168  183  ...      9     10      7   
1    0    0  141  253  247  239  235  237  234  233  ...      0      0      0   
2  123  124  126  128  130  131  133  135  139  142  ...     68     64     68   
3  237  237  237  244  254  252  235  198  186  188  ...    237    237    237   
4  255  255  255  255  255  255  255  255  255  254  ...     49     55     57   

   65530  65531  65532  65533  65534  65535      character  
0      7      5      6      7      5      6  Narendra Modi  
1      0      0      0      0      0      0  Narendra Modi  
2     74     78     80     79     79     78  Narendra Modi  
3    237    237    237    237    237    237  Narendra Modi  
4     55     53     52     47     45     49  Narendra Modi  

[5 rows x 65537 columns]


In [45]:
# saving the dataframe
if not os.path.exists('data'):
    os.mkdir('data')
#  Now we save the obtained dataframe as a csv file 
df_resultant.to_csv('data/eigenFace.csv',index = False)